In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt

 
plt.style.use(['science','notebook'])
plt.style.reload_library()

In [3]:
from june import World 
from june.geography import Geography
from june.demography import Demography
from june.interaction import Interaction
from june.infection import Infection, HealthIndexGenerator, InfectionSelector
from june.infection.transmission import TransmissionConstant
from june.groups import Hospitals, Schools, Companies, Households, CareHomes, Cemeteries, Universities
from june.groups.leisure import generate_leisure_for_config, Cinemas, Pubs, Groceries
from june.groups.travel import *
from june.simulator import Simulator
from june.infection_seed import InfectionSeed
from june.policy import Policy, Policies
from june import paths
from june.hdf5_savers import load_geography_from_hdf5
from june.records import Record, RecordReader

from june.world import generate_world_from_geography
from june.hdf5_savers import generate_world_from_hdf5

No --data argument given - defaulting to:
/home/florpi/JUNE/data
No --configs argument given - defaulting to:
/home/florpi/JUNE/configs


2020-10-01 15:43:23,483 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


# Initialize world

To initialize a certain world, we need to add the different components we want to have in it. First we specify what super areas (msoa) we want to create. We have included these ones, because they are known to contain hospitals, schools, care homes, and companies.

After creating the geography, we create the different components the worlds need to have such as care homes, companies ...

In [4]:
CONFIG_PATH = paths.configs_path / "config_example.yaml"

In [5]:
%%time 

geography = Geography.from_file(
{
    "super_area": ["E02001731", "E02002566","E02004935","E02000134", "E02004987"]
}
)

geography.hospitals = Hospitals.for_geography(geography)
geography.schools = Schools.for_geography(geography)
geography.companies = Companies.for_geography(geography)
geography.care_homes = CareHomes.for_geography(geography)
geography.universities = Universities.for_super_areas(geography.super_areas)
world = generate_world_from_geography(geography, include_households=True)


2020-10-01 15:43:24,043 - june.geography.geography - INFO - There are 151 areas and 5 super_areas and 3 in the world.
2020-10-01 15:43:24,048 - june.groups.hospital - INFO - There are 1 hospitals in this geography.
2020-10-01 15:43:24,109 - june.groups.school - INFO - There are 16 schools in this geography.
2020-10-01 15:43:24,151 - june.groups.school - INFO - No school for the age 0 in this world.
2020-10-01 15:43:24,157 - june.groups.school - INFO - No school for the age 1 in this world.
2020-10-01 15:43:24,505 - june.groups.carehome - INFO - There are 14 care_homes in this geography.
2020-10-01 15:43:24,566 - june.groups.university - INFO - There are 37 universities in this world.
2020-10-01 15:43:29,220 - june.world - INFO - Populating areas
2020-10-01 15:43:29,735 - june.world - INFO - Areas populated. This world's population is: 44314
2020-10-01 15:43:32,158 - june.distributors.worker_distributor - INFO - Distributing workers to super areas...
2020-10-01 15:43:33,571 - june.distr

## Commute, travel and leisure

In [6]:
%%time

world.pubs = Pubs.for_geography(geography)
world.cinemas = Cinemas.for_geography(geography)
world.groceries = Groceries.for_geography(geography)
leisure = generate_leisure_for_config(world, config_filename=CONFIG_PATH)
leisure.distribute_social_venues_to_areas(
    areas=world.areas, super_areas=world.super_areas
)


2020-10-01 15:43:43,802 - june.groups.leisure.social_venue - INFO - Initialized 800 pubs(s)
2020-10-01 15:43:43,815 - june.groups.leisure.social_venue - INFO - Initialized 8 cinemas(s)
2020-10-01 15:43:43,878 - june.groups.leisure.social_venue - INFO - Initialized 69 groceries(s)
2020-10-01 15:43:43,920 - june.groups.leisure.leisure - INFO - Linking households for visits
2020-10-01 15:43:44,411 - june.groups.leisure.leisure - INFO - Done
2020-10-01 15:43:44,412 - june.groups.leisure.leisure - INFO - Linking households with care homes for visits
2020-10-01 15:43:44,542 - june.groups.leisure.leisure - INFO - Done
2020-10-01 15:43:44,543 - june.groups.leisure.leisure - INFO - Distributing social venues to areas
2020-10-01 15:43:44,545 - june.groups.leisure.leisure - INFO - Distributed in 0 of 151 areas.
2020-10-01 15:43:44,650 - june.groups.leisure.leisure - INFO - Distributed in 151 of 151 areas.
CPU times: user 1.13 s, sys: 6.2 ms, total: 1.14 s
Wall time: 1.17 s


We are also going to need some cemeteries...geography.cemeteries = Cemeteries()


In [7]:
# initialise commuting
travel = Travel()
travel.initialise_commute(world)

2020-10-01 15:43:57,306 - june.groups.travel.travel - INFO - Initialising commute...
2020-10-01 15:43:57,309 - june.groups.travel.travel - INFO - Creating cities...
2020-10-01 15:43:57,319 - june.groups.travel.travel - INFO - This world has 4 cities, with names
['London', 'Newcastle upon Tyne', 'Darlington', 'St Albans']
2020-10-01 15:43:57,322 - june.groups.travel.travel - INFO - Creating stations...
2020-10-01 15:43:57,324 - june.groups.travel.travel - INFO - City London has 8 stations.
2020-10-01 15:43:57,326 - june.groups.travel.travel - INFO - City Newcastle upon Tyne has 4 stations.
2020-10-01 15:43:57,327 - june.groups.travel.travel - INFO - This world has 12 stations.
2020-10-01 15:43:57,328 - june.groups.travel.travel - INFO - Recording closest stations to super areas
2020-10-01 15:43:57,330 - june.groups.travel.travel - INFO - Determining people mode of transport
2020-10-01 15:44:05,437 - june.groups.travel.travel - INFO - Mode of transport allocated in 0 of 151 areas.
2020-1

In [8]:
world.cemeteries = Cemeteries()

In [9]:
len(world.people)

44314

### If it took a long time to run the previous commands, it might be a good idea to save the world to reuse it later.

In [10]:
world.to_hdf5("world.hdf5")

2020-10-01 15:44:05,911 - june.hdf5_savers.world_saver - INFO - saving world to HDF5
2020-10-01 15:44:05,948 - june.hdf5_savers.world_saver - INFO - saving population...
2020-10-01 15:44:07,163 - june.hdf5_savers.world_saver - INFO - saving hospitals...
2020-10-01 15:44:07,168 - june.hdf5_savers.world_saver - INFO - saving schools...
2020-10-01 15:44:07,175 - june.hdf5_savers.world_saver - INFO - saving companies...
2020-10-01 15:44:07,207 - june.hdf5_savers.world_saver - INFO - saving households...
2020-10-01 15:44:07,587 - june.hdf5_savers.world_saver - INFO - saving care homes...
2020-10-01 15:44:07,591 - june.hdf5_savers.world_saver - INFO - saving cities...
2020-10-01 15:44:07,597 - june.hdf5_savers.world_saver - INFO - saving stations...
2020-10-01 15:44:07,601 - june.hdf5_savers.world_saver - INFO - saving universities...
2020-10-01 15:44:07,607 - june.hdf5_savers.world_saver - INFO - saving social venues...


If we would like to load the world we saved, we just do

In [11]:
world = generate_world_from_hdf5("world.hdf5")

2020-10-01 15:44:07,655 - june.hdf5_savers.world_saver - INFO - loading world from HDF5
2020-10-01 15:44:07,669 - june.hdf5_savers.world_saver - INFO - loading hospitals...
2020-10-01 15:44:07,678 - june.hdf5_savers.world_saver - INFO - loading schools...
2020-10-01 15:44:07,686 - june.hdf5_savers.company_saver - INFO - loading companies...
2020-10-01 15:44:07,687 - june.hdf5_savers.company_saver - INFO - Companies chunk 0 of 1
2020-10-01 15:44:07,756 - june.hdf5_savers.world_saver - INFO - loading care homes...
2020-10-01 15:44:07,760 - june.hdf5_savers.world_saver - INFO - loading universities...
2020-10-01 15:44:07,765 - june.hdf5_savers.world_saver - INFO - loading cities...
2020-10-01 15:44:07,774 - june.hdf5_savers.world_saver - INFO - loading stations...
2020-10-01 15:44:07,778 - june.hdf5_savers.household_saver - INFO - loading households...
2020-10-01 15:44:07,780 - june.hdf5_savers.household_saver - INFO - Households chunk 0 of 1
2020-10-01 15:44:08,078 - june.hdf5_savers.pop

In [12]:
# and regenerate leisure in case we load it externally
leisure = generate_leisure_for_config(world, CONFIG_PATH)
# create travel as well
travel = Travel()

you have now a beautiful pre-pandemic world. 

# Adding the infection

The module in charge of infecting people is called the ``InfectionSelector``, which gives people a transmission time profile and a symptoms trajectory based on their age and sex (through the health index generator)

In [13]:
health_index_generator = HealthIndexGenerator.from_file(asymptomatic_ratio=0.2)
selector = InfectionSelector.from_file(
        health_index_generator=health_index_generator,
        transmission_config_path=paths.configs_path / 'defaults/transmission/XNExp.yaml'
)

# Adding the interaction

In [14]:
interaction = Interaction.from_file()

Beta are the intensities of the interaction taking place at the different groups

In [15]:
interaction.beta

{'box': 1,
 'pub': 0.42941,
 'grocery': 0.04137,
 'cinema': 0.157461,
 'city_transport': 0.107969,
 'inter_city_transport': 0.383,
 'hospital': 0.1168,
 'care_home': 0.28,
 'company': 0.371,
 'school': 0.07,
 'household': 0.208,
 'university': 0.306}

moreover this interaction module uses contact matrices, that are different for different groups. These contact matrices shouldnt be modified for now. However they are a combination of conversational contact matrices, and physical contact matrices (see the BBC pandemic paper, from where these matrices are extracted https://www.medrxiv.org/content/10.1101/2020.02.16.20023754v2)

There is a parameter, ``alpha`` ($\alpha$), that combines these two matrices in the following way,


$\beta M \left(1 + (\alpha -1) \right) P$

where $\beta$ is the intensity of the interaction, and $P$ the physical contact matrix. A larger $\alpha$ produces more physical contacts. It is an overall number, non dependent of the particular group.


In [16]:
interaction.alpha_physical

2.0

# Seed the disease

There are two options implemented in the seed at the moment, either you specify the number of cases and these are then homogeneously distributed by population to the different areas, or you use UK data on cases per region. For now use the first case.

In [17]:
infection_seed = InfectionSeed(
    world, selector,
)

In [18]:
n_cases = 50
infection_seed.unleash_virus(
    population=world.people,
    n_cases=n_cases) # play around with the initial number of cases

# Set policies

In [19]:
policies = Policies.from_file()

In [20]:
policies

# Run the simulation

The simulator is the main module in charge of running the simulation. It coordinates the ``ActivityManager`` which is responsible of allocating people to the right groups given the current timestep, it updates the health status of the population, and it runs the interaction over the different groups. All of these modules can be modified by policies at any given time.

Since the timer configuration is a bit cumbersome, it is read from the config file at ``configs/config_example.yaml``

In [21]:
record = Record(    
    record_path = 'results',    
    record_static_data=True,
) 

In [22]:
record.static_data(world=world)

In [23]:
len(world.pubs)

800

In [24]:
simulator = Simulator.from_file(
    world=world,
    infection_selector=selector,
    interaction=interaction, 
    config_filename = CONFIG_PATH,
    leisure = leisure,
    travel = travel,
    record=record,
    policies = policies
)

In [ ]:
%%time
simulator.run()

2020-10-01 15:44:11,614 - june.simulator - INFO - Starting simulation for 60 days at day 2020-03-01 00:00:00, to run for 60 days
2020-10-01 15:44:11,971 - june.activity.activity_manager - INFO - CMS: People COMS for rank 0/1 - 4.404006176628172e-06,3.814697265625e-06 - 2020-03-01 00:00:00
2020-10-01 15:44:11,989 - june.simulator - INFO - Date = 2020-03-01 00:00:00, number of deaths =  0, number of infected = 50
2020-10-01 15:44:12,126 - june.simulator - INFO - CMS: Infection COMS for rank 0/1 - 6.696005584672093e-06,5.9604644775390625e-06 - 2020-03-01 00:00:00
2020-10-01 15:44:12,287 - june.simulator - INFO - CMS: Timestep for rank 0/1 - 0.6161678930075141, 0.6161675453186035 - 2020-03-01 00:00:00
2020-10-01 15:44:12,610 - june.activity.activity_manager - INFO - CMS: People COMS for rank 0/1 - 3.7669960875064135e-06,3.337860107421875e-06 - 2020-03-01 04:00:00
2020-10-01 15:44:12,617 - june.simulator - INFO - Date = 2020-03-01 04:00:00, number of deaths =  0, number of infected = 50
202

2020-10-01 15:44:19,482 - june.activity.activity_manager - INFO - CMS: People COMS for rank 0/1 - 6.132002454251051e-06,5.245208740234375e-06 - 2020-03-04 00:00:00
2020-10-01 15:44:19,491 - june.simulator - INFO - Date = 2020-03-04 00:00:00, number of deaths =  0, number of infected = 88
2020-10-01 15:44:19,815 - june.simulator - INFO - CMS: Infection COMS for rank 0/1 - 1.5394005458801985e-05,1.1682510375976562e-05 - 2020-03-04 00:00:00
2020-10-01 15:44:19,953 - june.simulator - INFO - CMS: Timestep for rank 0/1 - 0.621415835004882, 0.6214165687561035 - 2020-03-04 00:00:00
2020-10-01 15:44:20,298 - june.activity.activity_manager - INFO - CMS: People COMS for rank 0/1 - 1.0200004908256233e-05,8.58306884765625e-06 - 2020-03-04 01:00:00
2020-10-01 15:44:20,307 - june.simulator - INFO - Date = 2020-03-04 01:00:00, number of deaths =  0, number of infected = 88
2020-10-01 15:44:21,037 - june.simulator - INFO - CMS: Infection COMS for rank 0/1 - 5.2170071285218e-06,4.291534423828125e-06 - 2

2020-10-01 15:44:27,880 - june.simulator - INFO - Date = 2020-03-06 13:00:00, number of deaths =  0, number of infected = 236
2020-10-01 15:44:28,039 - june.simulator - INFO - CMS: Infection COMS for rank 0/1 - 3.2609968911856413e-06,2.6226043701171875e-06 - 2020-03-06 13:00:00
2020-10-01 15:44:28,122 - june.simulator - INFO - CMS: Timestep for rank 0/1 - 0.3424146710021887, 0.34241557121276855 - 2020-03-06 13:00:00
2020-10-01 15:44:28,419 - june.activity.activity_manager - INFO - CMS: People COMS for rank 0/1 - 4.569999873638153e-06,3.814697265625e-06 - 2020-03-07 00:00:00
2020-10-01 15:44:28,426 - june.simulator - INFO - Date = 2020-03-07 00:00:00, number of deaths =  0, number of infected = 251
2020-10-01 15:44:28,582 - june.simulator - INFO - CMS: Infection COMS for rank 0/1 - 3.096996806561947e-06,2.384185791015625e-06 - 2020-03-07 00:00:00
2020-10-01 15:44:28,676 - june.simulator - INFO - CMS: Timestep for rank 0/1 - 0.5535198300058255, 0.5535211563110352 - 2020-03-07 00:00:00
20

While the simulation runs (and afterwards) we can launch the visualization webpage by running
```python june/visualizer.py path/to/results``` 

# Getting the results

All results are stored in a json file specified in the ``record.record_path`` folder. Summaries are found under ``summary.csv``

In [ ]:
import pandas as pd

In [ ]:
read = RecordReader()

## Contains summaries with regional information

In [ ]:
read.regional_summary.head(3)

In [ ]:
for region in read.regional_summary['region'].unique():
    read.regional_summary[
        read.regional_summary['region'] == region
    ]['current_infected'].plot()
    read.regional_summary[
        read.regional_summary['region'] == region
    ]['current_susceptible'].plot()
    plt.title(region)
    plt.show()

In [ ]:
read.world_summary['current_infected'].plot()
read.world_summary['current_susceptible'].plot()

# Asking questions to the records

## Sero-prevalence by age

In [ ]:
infections_df = read.get_table_with_extras('infections',
                                           'infected_ids')

In [ ]:
deaths_df = read.get_table_with_extras('deaths', 
                                       'dead_person_ids')

In [ ]:
age_bins = (0,20,60,100)

In [ ]:
infected_by_age = infections_df.groupby([pd.cut(infections_df['age'],
            bins=age_bins), 'timestamp']).size()

In [ ]:
people_df = read.table_to_df('population')

In [ ]:
n_by_age = people_df.groupby(pd.cut(people_df['age'],
            bins=age_bins)).size()

In [ ]:
(100*infected_by_age/n_by_age).xs(10).cumsum().plot(label='0,20')
(100*infected_by_age/n_by_age).xs(30).cumsum().plot(label='20,60')
(100*infected_by_age/n_by_age).xs(70).cumsum().plot(label='60,100')
plt.legend()

## Care home deaths in hospital

In [ ]:
care_home_deaths_hospital = deaths_df[
    (deaths_df['location_specs'] == 'hospital') 
    & (deaths_df['residence_type'] == 'care_home')
]
care_home_deaths_hospital=care_home_deaths_hospital.groupby(
    ['name_region', 'timestamp']
).size()

In [ ]:
care_home_deaths_hospital.unstack(level=0).plot()


## Where people get infected as a function of time

In [ ]:
locations_df = infections_df.groupby(['location_specs', 
                                'timestamp']).size()

In [ ]:
locations_df.unstack(level=0).plot()

In [ ]:
import matplotlib.ticker as mtick
location_counts_df = locations_df.groupby('location_specs').size()
location_counts_df = 100*location_counts_df / location_counts_df.sum()
ax = location_counts_df.sort_values().plot.bar()
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel('Percentage of infections at location')
plt.xlabel('location')


## Where people of certain age get infected as a function of time

In [ ]:
old_locations_df = infections_df[
    infections_df.age > 65
].groupby(['location_specs', 'timestamp']).size()

In [ ]:
old_locations_df.unstack(level=0).plot()

## Prevalence by household size

In [ ]:
household_people = people_df[
    people_df['residence_type'] == 'household'
]

In [ ]:
household_sizes = household_people.groupby('residence_id').size()

In [ ]:
household_sizes.hist() # in units of households

In [ ]:
household_people.loc[:,'household_size'] = household_sizes.loc[
    household_people['residence_id']
].copy(deep=True).values

In [ ]:
household_people['household_size'].hist() # in units of people

In [ ]:
household_infections_df = infections_df.merge(
    household_people['household_size'], 
    left_index=True, right_index=True, how='inner'
)

In [ ]:
(household_infections_df.groupby(
    'household_size'
).size()/household_people.groupby('household_size').size()).plot()
plt.xlabel('Household size')
plt.ylabel('% of people infected by household size')
plt.xlim(0,8)

In [ ]:
# How many households have everyone infected?

In [ ]:
n_infected_by_household = infections_df[
    infections_df['residence_type'] == 'household'
].groupby('residence_id').size()

In [ ]:
n_total_in_household = household_people[
    household_people['residence_id'].isin(
        n_infected_by_household.index
    )
].groupby('residence_id').size()

In [ ]:
(n_infected_by_household/n_total_in_household).hist()
plt.xlabel('% of the household infected')

## Percentage of infected per care home

In [ ]:
n_infected_by_carehome = infections_df[
    infections_df['residence_type'] == 'care_home'
].groupby(
    'residence_id'
).size()

In [ ]:
n_total_in_carehome = people_df[
    (people_df['residence_type'] == 'care_home') 
    & (people_df['residence_id'].isin(n_infected_by_carehome.index))  
].groupby('residence_id').size()

In [ ]:
(n_infected_by_carehome/n_total_in_carehome).hist()
plt.xlabel('% of the care home infected')

In [ ]:
# from all care homes, how many got at least one case?

In [ ]:
n_total_care_homes = people_df[
    (people_df['residence_type'] == 'care_home') 
]['residence_id'].nunique()

In [ ]:
n_total_care_homes

In [ ]:
care_homes_with_infected = infections_df[
    (infections_df['residence_type'] == 'care_home') 
]['residence_id'].nunique()

In [ ]:
care_homes_with_infected/n_total_care_homes